In this walkthrough we will build a full text search capable application by incrementally building the features to be exposed via a simple REST API.

## Install Prerequisites

In [24]:
! pip install "pymongo[srv]"


You should consider upgrading via the '/Users/tom.mccarthy/Projects/atlas-search-guide/.venv/bin/python -m pip install --upgrade pip' command.


In [25]:
# Just making sure everything prints cleanly

from pygments.style import Style
from pygments.token import Token
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import Terminal256Formatter


class MyStyle(Style):
    styles = {
        Token.String: 'ansigreen',
        Token.Literal: 'ansibrightyellow',
        Token.Keyword: 'ansimagenta',
        Token.Operator: 'ansibrightmagenta'
    }

def pp(doc):
    formatted_json = json.dumps(json.loads(json_util.dumps(doc)), indent=4)
    colorful_json = highlight(formatted_json, JsonLexer(), Terminal256Formatter(style=MyStyle))
    print(colorful_json)

In [26]:
import pymongo
from bson import json_util
from config import mongo_uri
import json
    
conn = pymongo.MongoClient(mongo_uri)

movies_collection = conn['sample_mflix']['movies']

## Basic Search

Run a simple text search.

In [28]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "fight",
                'path': "title"
            }
        }
    },
    {
        '$project': {
            'title':1,
            '_id':0,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]
docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887
    },
    {
        "title": "The Fighting Lady",
        "score": 3.4211220741271973
    },
    {
        "title": "Fight Club",
        "score": 3.4211220741271973
    },
    {
        "title": "The Fighting Temptations",
        "score": 3.4211220741271973
    },
    {
        "title": "Street Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "Girl Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "A Fighting Man",
        "score": 3.4211220741271973
    },
    {
        "title": "Fight Back to School",
        "score": 2.8717684745788574
    },
    {
        "title": "Why We Fight",
        "score": 2.8717684745788574
    },
    {
        "title": "Love at First Fight",
        "scor

## Fuzzy

Often referred to as approximate string matching, fuzzy matching is a technique of finding strings that match a pattern approximately rather than exactly. It's common use case is when there are common mispellings, when users make errors ("fat fingering"), etc.

**maxEdits** uses the Levenshtein distance, which is the difference between two string sequences.

In [18]:
pipeline = [
    {
        '$search': {
            'text': {
                'query': "might cub",
                'path': "title",
                'fuzzy':{
                    'maxEdits':2
                }
            }
        }
    },
    {
        '$project': {
            'title':1,
            '_id':0,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]
docs = movies_collection.aggregate(pipeline)
pp(docs)


[
    {
        "title": "Fight Club",
        "score": 3.7854979038238525
    },
    {
        "title": "Cub",
        "score": 3.6564555168151855
    },
    {
        "title": "Knight of Cups",
        "score": 3.331955909729004
    },
    {
        "title": "Fright Night",
        "score": 3.1747939586639404
    },
    {
        "title": "Fright Night",
        "score": 3.1747939586639404
    },
    {
        "title": "Eight Crazy Nights",
        "score": 3.045708179473877
    },
    {
        "title": "Friday Night Lights",
        "score": 3.045708179473877
    },
    {
        "title": "The High and the Mighty",
        "score": 2.9480228424072266
    },
    {
        "title": "Monster High: Friday Night Frights",
        "score": 2.7375903129577637
    },
    {
        "title": "Fight for Your Right Revisited",
        "score": 2.62430477142334
    },
    {
        "title": "Eight Miles High",
        "score": 2.474637985229492
    },
    {
        "title": "The Club",
        

## Highlighting

Add a relevance score and hit highlights to the results.

In [6]:
pipeline = [
        {
            '$search': {
                'text': {
                    'query': "fight",
                    'path': "title"
                },
                # text highlighting
                'highlight': { "path": "title" }
            }
        }, {
            '$project': {
                'title':1,
                '_id':0,
                'score': {
                    '$meta': 'searchScore'
                },
                'highlights': {"$meta": "searchHighlights"},
                'score': {
                    '$meta': 'searchScore'
                }
            }
        }
    ]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355,
        "highlights": [
            {
                "score": 1.408543348312378,
                "path": "title",
                "texts": [
                    {
                        "value": "Fighting",
                        "type": "hit"
                    }
                ]
            }
        ]
    },
    {
        "title": "Fight, Zatoichi, Fight",
        "score": 4.106432914733887,
        "highlights": [
            {
                "score": 1.1319897174835205,
                "path": "title",
                "texts": [
                    {
                        "value": "Fight",
                        "type": "hit"
                    },
                    {
                        "value": ", Zatoichi, ",
                        "type": "text"
                    },
                    {
                        "value": "Fight",
                        "type": "hit"
                    

## Autocomplete

Edit the "query" field to try different searches

In [7]:
pipeline = [
    {
        '$search': {
            'index': 'autocomplete',
            'autocomplete': {
                'query': "fl",
                'path': "title"
            }
        }
    },
    {
        '$project': {
            'title': 1,
            'score': {
                    '$meta': 'searchScore'
                },
            '_id': 0
        }
    },
    {
        '$limit': 5
    },
    {
        '$sort': {'score': -1}
    }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "The Flim-Flam Man",
        "score": 4.277243614196777
    },
    {
        "title": "Flower Drum Song",
        "score": 4.10808801651001
    },
    {
        "title": "Flatfoot in Africa",
        "score": 4.10808801651001
    },
    {
        "title": "Flodder in Amerika!",
        "score": 4.10808801651001
    },
    {
        "title": "Flesh and Bone",
        "score": 4.10808801651001
    }
]



# Compound

Compound queries grant the ability to add multiple conditions to your search query. Each element is a clause, where you can embed sub-queries.

In [8]:
pipeline = [
    {
        "$search": {
            "compound": {
                "must": [{"text": {"query": "Fight", "path": "title"}}],
                "mustNot": [
                    {"text": {"query": "Zatoichi", "path": "title"}},
                    {"text": {"query": "Club", "path": "title"}},
                ],
            }
        }
    },
    {"$project": {"title": 1, "score": {"$meta": "searchScore"}, "_id": 0}},
    {"$limit": 5},
    {"$sort": {"score": -1}},
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fighting",
        "score": 4.2303690910339355
    },
    {
        "title": "The Fighting Lady",
        "score": 3.4211220741271973
    },
    {
        "title": "The Fighting Temptations",
        "score": 3.4211220741271973
    },
    {
        "title": "Street Fight",
        "score": 3.4211220741271973
    },
    {
        "title": "Girl Fight",
        "score": 3.4211220741271973
    }
]



# Facets

In [9]:
from dateutil import parser

pipeline = [
 {
   "$searchMeta": {
     "index": "facet_index",
     "facet": {
       "operator": {
         "range": {
           "path": "released",
           "gte": parser.parse("2000-01-01T00:00:00.000Z"),
           "lte": parser.parse("2015-01-31T00:00:00.000Z")
         }
       },
       "facets": {
         "directorsFacet": {
           "type": "string",
           "path": "directors",
           "numBuckets" : 7
         },
         "yearFacet" : {
           "type" : "number",
           "path" : "year",
           "boundaries" : [2000,2005,2010,2015]
         }
       }
     }
   }
 }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "count": {
            "lowerBound": 13064
        },
        "facet": {
            "yearFacet": {
                "buckets": [
                    {
                        "_id": 2000,
                        "count": 3283
                    },
                    {
                        "_id": 2005,
                        "count": 4365
                    },
                    {
                        "_id": 2010,
                        "count": 5123
                    }
                ]
            },
            "directorsFacet": {
                "buckets": [
                    {
                        "_id": "Takashi Miike",
                        "count": 29
                    },
                    {
                        "_id": "Johnnie To",
                        "count": 23
                    },
                    {
                        "_id": "Steven Soderbergh",
                        "count": 21
                    },
              

# Synonyms

Requires index set up with synonyms for title field. 

In [10]:
pipeline = [
  {
    '$search': {
      'text': {
        'path': 'title',
        'query': 'automobile',
        'synonyms': 'transportSynonyms'
      }
    }
  }, 
  {
    '$limit': 10
  }, 
  {
    '$project': {
      '_id': 0,
      'title': 1,
      'score': {
        '$meta': 'searchScore'
      }
    }
  }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Cars",
        "score": 4.197734832763672
    },
    {
        "title": "Planes, Trains & Automobiles",
        "score": 3.8511905670166016
    },
    {
        "title": "Car Wash",
        "score": 3.39473032951355
    },
    {
        "title": "Used Cars",
        "score": 3.39473032951355
    },
    {
        "title": "Blue Car",
        "score": 3.39473032951355
    },
    {
        "title": "Cars 2",
        "score": 3.39473032951355
    },
    {
        "title": "Stealing Cars",
        "score": 3.39473032951355
    },
    {
        "title": "Cop Car",
        "score": 3.39473032951355
    },
    {
        "title": "The Cars That Eat People",
        "score": 2.8496146202087402
    },
    {
        "title": "Khrustalyov, My Car!",
        "score": 2.8496146202087402
    }
]



In [11]:
pipeline = [
    {
        '$search': {
            'compound': {
                'should': [
                    {
                        'text': {
                            'path': 'title',
                            'query': 'boat',
                            'synonyms': 'transportSynonyms'
                        }
                    }, {
                        'text': {
                            'path': 'title',
                            'query': 'hat',
                            'synonyms': 'attireSynonyms'
                        }
                    }
                ]
            }
        }
    }, {
        '$limit': 10
    }, {
        '$project': {
            '_id': 0,
            'title': 1,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
]

docs = movies_collection.aggregate(pipeline)
pp(docs)

[
    {
        "title": "Fedora",
        "score": 5.673145294189453
    },
    {
        "title": "Vessel",
        "score": 5.373150825500488
    },
    {
        "title": "Boats",
        "score": 4.589139938354492
    },
    {
        "title": "And the Ship Sails On",
        "score": 4.3452959060668945
    },
    {
        "title": "Broken Vessels",
        "score": 4.3452959060668945
    },
    {
        "title": "Sailing to Paradise",
        "score": 4.3452959060668945
    },
    {
        "title": "Top Hat",
        "score": 4.066137313842773
    },
    {
        "title": "A Hatful of Rain",
        "score": 4.066137313842773
    },
    {
        "title": "Boat People",
        "score": 3.711261749267578
    },
    {
        "title": "Boat Trip",
        "score": 3.711261749267578
    }
]



In [12]:
conn.close()